In [2]:
import dicom2nifti
import os 
import pydicom
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
from dcmrtstruct2nii import dcmrtstruct2nii, list_rt_structs
import shutil 
import pandas as pd


Please cite:
Thomas Phil, Thomas Albrecht, Skylar Gay, & Mathis Ersted Rasmussen. (2023). Sikerdebaard/dcmrtstruct2nii: dcmrtstruct2nii v5 (Version v5). Zenodo. https://doi.org/10.5281/zenodo.4037864



# Extract RT STUCT files to nii

## Load RT STRUCT

## Organize folders 

In [2]:
clean_path = "/home/tachennf/Documents/delta-rad/data/Madrid/clean_dicom_data/"
if os.path.exists(clean_path) == False: 
    os.mkdir(clean_path)

patients_list = os.listdir("/home/tachennf/Documents/delta-rad/data/Madrid/dicom_data/")
for p in patients_list:
    print(p)
    patient_folder = "/home/tachennf/Documents/delta-rad/data/Madrid/dicom_data/" + p + "/"
    # récupérer tous les folders (image, rtstruct, rtdose, rtplan)
    dicom_folders = [x for x in os.listdir(patient_folder) if "_MR_" in x]
    rt_struct_folders = [x for x in os.listdir(patient_folder) if "RTst" in x]
    rt_dose_folders = [x for x in os.listdir(patient_folder) if "RTDOSE" in x]
    rt_plan_folders = [x for x in os.listdir(patient_folder) if "RTPLAN" in x]

    if p == 'Mont16':
        # remove folder that contain "Pancreas2_n1__00000" in the name
        dicom_folders = [x for x in dicom_folders if "Pancreas2_n1__00000" not in x]
        rt_struct_folders = [x for x in rt_struct_folders if "Pancreas2_n1__00000" not in x]
        rt_dose_folders = [x for x in rt_dose_folders if "Pancreas2_n1__00000" not in x]
        rt_plan_folders = [x for x in rt_plan_folders if "Pancreas2_n1__00000" not in x]
        assert len(dicom_folders) == len(rt_struct_folders) == len(rt_dose_folders) == len(rt_plan_folders), "Different number of folders"

    # creer un dossier pour chaque patient dans clean_data
    if os.path.exists(clean_path + p) == False:
        os.mkdir(clean_path + p)

    for i in range(len(dicom_folders)):
        if 'F0' in dicom_folders[i]:
            serie_name = 'F1'
        elif 'F1' in dicom_folders[i]:
            serie_name = 'F2'
        elif 'F2' in dicom_folders[i]:
            serie_name = 'F3'
        elif 'F3' in dicom_folders[i]:
            serie_name = 'F4'
        elif 'F4' in dicom_folders[i]:
            serie_name = 'F5'
        else: 
            serie_name = 'SIMU'

        if os.path.exists(clean_path + p + "/" + serie_name) == False:
            os.mkdir(clean_path + p + "/" + serie_name) # creer un dossier pour chaque série

        dicom_folder = patient_folder + dicom_folders[i]
        rt_struct_folder = patient_folder + rt_struct_folders[i]
        rt_dose_folder = patient_folder + rt_dose_folders[i]
        rt_plan_folder = patient_folder + rt_plan_folders[i]
#  
        # # creer un dossier pour chaque modalité 
        if os.path.exists(clean_path + p + "/" + serie_name + "/image") == False:
            os.mkdir(clean_path + p + "/" + serie_name + "/image")
        if os.path.exists(clean_path + p + "/" + serie_name + "/rt_struct") == False:
            os.mkdir(clean_path + p + "/" + serie_name + "/rt_struct")
        if os.path.exists(clean_path + p + "/" + serie_name + "/rt_dose") == False:
            os.mkdir(clean_path + p + "/" + serie_name + "/rt_dose")
        if os.path.exists(clean_path + p + "/" + serie_name + "/rt_plan") == False:
            os.mkdir(clean_path + p + "/" + serie_name + "/rt_plan")
        #     
        # copier les dicom depuis l'ancien dossier vers le nouveau 
        for file in os.listdir(dicom_folder):
            shutil.copy(dicom_folder + '/' + file, clean_path + p + "/" + serie_name + "/image") 
        # copier les rtstruct depuis l'ancien dossier vers le nouveau
        for file in os.listdir(rt_struct_folder):
            shutil.copy(rt_struct_folder + '/' + file, clean_path + p + "/" + serie_name + "/rt_struct") 
        # copier les rtdose depuis l'ancien dossier vers le nouveau
        for file in os.listdir(rt_dose_folder):
            shutil.copy(rt_dose_folder + '/' + file, clean_path + p + "/" + serie_name + "/rt_dose")
        # copier les rtplan depuis l'ancien dossier vers le nouveau
        for file in os.listdir(rt_plan_folder):
            shutil.copy(rt_plan_folder + '/' + file, clean_path + p + "/" + serie_name + "/rt_plan")


Mont10
Mont1
Mont32
Mont23
Mont35
Mont24
Mont33
Mont18
Mont9
Mont2
Mont6
Mont16
Mont15
Mont13
Mont4
Mont29
Mont19
Mont26
Mont31
Mont11
Mont27
Mont8
Mont37
Mont34
Mont14
Mont28
Mont7
Mont5
Mont21
Mont25
Mont3
Mont17
Mont20
Mont30
Mont36
Mont22
Mont12


Organiser dossier en: 
patients 
    date 
        image
        rt_dose 
        rt_struct 
        rt_plan

## Generate structure to extract csv file 

In [ ]:
clean_data_folder = "/home/tachennf/Documents/delta-rad/data/Madrid/clean_dicom_data/"

patients_list = os.listdir(clean_data_folder)
structure_count = {}
issue_patients = []

for p in patients_list: 
    series = os.listdir(clean_data_folder + p)
    for s in series: 
        rt_struct_folder = clean_data_folder + p + '/' + s + '/rt_struct/'

        rt_struct_path = rt_struct_folder + os.listdir(rt_struct_folder)[0]
        list_rt_struct = list_rt_structs(rt_struct_path)

        structures = []
        if "GTV" in list_rt_struct: 
            structures.append("GTV")
        if "PTV" in list_rt_struct: 
            structures.append("PTV")
#   
        if len(structures) != 2: 
            issue_patients.append(p)
            print(p, list_rt_struct)
            continue 


Mont25 ['Skin', 'AGUA', 'AIRE', 'ANILLO1MENOS05CM', 'ANILLO5MENOS3', 'Aorta', 'Bowel', 'CTV 35', 'CTV 50', 'CRITICAL_ALL', 'CRITICAL_IDF', 'CRITICAL_STRUCTU', 'Duodenum', 'EXP05CM', 'EXP15CM', 'EXP3CM', 'EXP5CM', 'Kidney_L', 'Kidney_R', 'Liver', 'PTV35', 'PTV35_PTV50', 'PTV35_H', 'PTV35_L', 'PTV50', 'PTV50_H', 'PTV50_L', 'SpinalCord', 'Stomach', 'V_CavaInferior', 'Isocenter']
Mont25 ['Skin', 'AGUA', 'AIRE', 'ANILLO1MENOS05CM', 'ANILLO5MENOS3', 'Aorta', 'Bowel', 'CTV 35', 'CTV 50', 'CRITICAL_ALL', 'CRITICAL_IDF', 'CRITICAL_STRUCTU', 'Duodenum', 'EXP05CM', 'EXP15CM', 'EXP3CM', 'EXP5CM', 'Kidney_L', 'Kidney_R', 'Liver', 'PTV35', 'PTV35_PTV50', 'PTV35_H', 'PTV35_L', 'PTV50', 'PTV50_H', 'PTV50_L', 'SpinalCord', 'Stomach', 'V_CavaInferior', 'Isocenter']
Mont25 ['Skin', 'AGUA', 'AIRE', 'ANILLO1MENOS05CM', 'ANILLO5MENOS3', 'Aorta', 'Bowel', 'CTV 35', 'CTV 50', 'CRITICAL_ALL', 'CRITICAL_IDF', 'CRITICAL_STRUCTU', 'Duodenum', 'EXP05CM', 'EXP15CM', 'EXP3CM', 'EXP5CM', 'Kidney_L', 'Kidney_R', 'Live

## Write simple patients in csv

In [14]:
issue_df = pd.read_csv("/home/tachennf/Documents/delta-rad/data/Madrid/structures_to_extract.csv")
for p in patients_list: 
    # add the patient to the csv file
    if p not in issue_df['Patient_name'].values:
        if p in issue_patients:
            new_row = pd.DataFrame({'Patient_name': p, 'GTV': '', 'PTV': ''}, index=[0])
        else: 
            new_row = pd.DataFrame({'Patient_name': p, 'GTV': 'GTV', 'PTV': 'PTV'}, index=[0])
        issue_df = pd.concat([issue_df, new_row], ignore_index=True)


In [15]:
issue_df

,Patient_name,GTV,PTV
0,Mont10,GTV,PTV
1,Mont32,GTV,PTV
2,Mont23,GTV,PTV
3,Mont35,GTV,PTV
4,Mont24,GTV,PTV
5,Mont33,GTV,PTV
6,Mont18,GTV,PTV
7,Mont6,GTV,PTV
8,Mont16,GTV,PTV
9,Mont26,GTV,PTV


In [16]:
# save as csv file
issue_df.to_csv("/home/tachennf/Documents/delta-rad/data/Madrid/structures_to_extract.csv", index=False)

The other patients will have their GTV and PTV inserted manually according to radiotherapist advice. 

## Convert image and rt_struct to nifti

In [4]:
structure_csv_path = "/home/tachennf/Documents/delta-rad/data/Madrid/structures_to_extract_full.csv"
structure_df = pd.read_csv(structure_csv_path)
structure_df.head()


,Patient_name,GTV,PTV
0,Mont10,GTV,PTV
1,Mont32,GTV,PTV
2,Mont23,GTV,PTV
3,Mont35,GTV,PTV
4,Mont24,GTV,PTV


In [5]:
# check for nan values
structure_df.isnull().sum()

Patient_name    0
GTV             0
PTV             0
dtype: int64

In [8]:
clean_data_folder = "/home/tachennf/Documents/delta-rad/data/Madrid/clean_dicom_data/"
nifti_folder = "/home/tachennf/Documents/delta-rad/data/Madrid/nifti_data/"

if os.path.exists(nifti_folder) == False: 
    os.mkdir(nifti_folder)

patients_list = os.listdir(clean_data_folder)
for p in patients_list: 
    if os.path.exists(nifti_folder + p) == False: 
        os.mkdir(nifti_folder + p)
    else: 
        continue
    series = os.listdir(clean_data_folder + p)

    # get structures from csv file 
    gtv_name = structure_df.loc[structure_df['Patient_name'] == p, 'GTV'].values[0]
    ptv_name = structure_df.loc[structure_df['Patient_name'] == p, 'PTV'].values[0]
    structure_names = [gtv_name, ptv_name]

    for s in series: 
        # convert dicom to nifti (images and structures)
        if os.path.exists(nifti_folder + p + '/' + s) == False: 
            os.mkdir(nifti_folder + p + '/' + s)
        image_folder = clean_data_folder + p + '/' + s + '/image/'
        rt_struct_folder = clean_data_folder + p + '/' + s + '/rt_struct/'
        rt_struct_path = rt_struct_folder + os.listdir(rt_struct_folder)[0]        
        output_path = nifti_folder + p + '/' + s + '/'
        dcmrtstruct2nii(rt_struct_path, image_folder, output_path, structures=structure_names, mask_foreground_value=1)

        if p == 'Mont25':
            os.rename(output_path + 'mask_CTV-50.nii.gz', output_path + 'mask_GTV.nii.gz')
            os.rename(output_path + 'mask_PTV50.nii.gz', output_path + 'mask_PTV.nii.gz')
        else: 
            # Rename file to GTV and PTV 
            if structure_names[0] != 'GTV':
                os.rename(output_path + 'mask_' + str(structure_names[0]) + '.nii.gz', output_path + 'mask_GTV.nii.gz')
            if structure_names[1] != 'PTV':
                os.rename(output_path + 'mask_' + str(structure_names[1]) + '.nii.gz', output_path + 'mask_PTV.nii.gz')
